In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
patient = pd.read_csv('MIBI Image Data/Detections_OP_P1.csv')
patient.head()

,Object ID,Name,Class,Parent,Centroid X µm,Centroid Y µm
0,f0a0f6ee-bf8c-4cf4-821b-13abce782117,PathCellObject,NaN,Tumor,596.71,5.279
1,2d2d4733-7bb3-4ad9-8248-2e1d92944dfa,PathCellObject,NaN,Tumor,567.48,5.257
2,8a2266f1-fe18-410c-b334-65e6e2fb6ed4,PathCellObject,NaN,Tumor,767.00,2.690
3,df48b9ac-cc49-4a75-9f2d-49d59c40b066,Target: CD3: CD4,Target: CD3: CD4,Tumor,751.92,7.052
4,b06f4b65-b938-4b3f-b93c-65537b8e5078,PathCellObject,NaN,Tumor,60.60,5.255


In [3]:
patient = patient[["Class", "Parent", "Centroid X µm", "Centroid Y µm"]]

In [4]:
patient

,Class,Parent,Centroid X µm,Centroid Y µm
0,NaN,Tumor,596.71,5.279
1,NaN,Tumor,567.48,5.257
2,NaN,Tumor,767.00,2.690
3,Target: CD3: CD4,Tumor,751.92,7.052
4,NaN,Tumor,60.60,5.255
...,...,...,...,...
3930,Target: CD20,Other,496.54,796.810
3931,NaN,Other,609.44,796.590
3932,NaN,Other,666.72,797.820
3933,NaN,Other,677.15,797.840


In [5]:
patient['Class'] = patient['Class'].fillna(patient['Parent'])

In [6]:
patient = patient.drop(['Parent'], axis=1)

In [7]:
patient['Class'] = patient['Class'].apply(lambda x: x.replace('Target: ', ''))

In [8]:
patient['Class'] = patient['Class'].apply(lambda x: x.replace(': ', '+'))

In [13]:
mapping = {
    'CD3+CD4':'Helper T cell',
    'CD8':'Killer T cell',
    'CD20':'B cell',
    'CD3':'T cell',
    'CD68':'Macrophage',
    'CD3+CD8':'Killer T cell',
    'CD4':'Helper T cell',
    'FoxP3':'Regulatory T cell',
    'CD3+FoxP3':'Regulatory T cell',
    'CD3+CD4+FoxP3': 'Regulatory T cell',
    'CD3+CD8+FoxP3': 'Regulatory T cell',
    'CD3+CD4+CD8':'CD4+CD8'
}

patient['Class'] = patient['Class'].map(lambda x: mapping.get(x, f'{x}'))

In [10]:
new_rows = {
    'Class': [], 
    'Centroid X µm': [],
    'Centroid Y µm': []
}

for idx, row in patient.iterrows():
    if 'CD20' in list(row['Class'].split('+')) or 'CD68' in list(row['Class'].split('+')):
        for clas_ in list(row['Class'].split('+')):
            new_rows['Class'].append(clas_)

        for _ in range(len(list(row['Class'].split('+')))):
            new_rows['Centroid X µm'].append(row['Centroid X µm'])
            new_rows['Centroid Y µm'].append(row['Centroid Y µm']) 
        
        patient = patient.drop(idx)  # type: ignore



new_rows = pd.DataFrame(new_rows)
patient = pd.concat([patient, new_rows], ignore_index=True)       

In [11]:
patient

,Class,Centroid X µm,Centroid Y µm
0,Tumor,596.71,5.279
1,Tumor,567.48,5.257
2,Tumor,767.00,2.690
3,Helper T cell,751.92,7.052
4,Tumor,60.60,5.255
...,...,...,...
4136,CD20,275.04,774.770
4137,CD3,275.04,774.770
4138,CD20,611.30,776.370
4139,CD3,611.30,776.370


In [14]:
patient['Class'].unique()

array(['Tumor', 'Helper T cell', 'Killer T cell', 'B cell', 'T cell',
       'Macrophage', 'Other', 'CD4+CD8'], dtype=object)

Split into 2 cells when overlapping
CD20: B cell
CD68: Macrophage

CD3: T cell
CD4: Helper T cell
CD8: Killer T cell
FoxP3: Regulatory T cell
CD3 + CD4: Helper T cell
CD3 + CD8: Killer T cell
CD3 + FoxP3: Regulatory T cell
CD3 + CD4 + FoxP3: Regulatory T cell
CD3 + CD8 + FoxP3: Regulatory T cell

CD3 + CD4 + CD8: Ignore CD3 then classify into helper and killer cells after splitting

Not possible, split into two 
CD4       +    CD8 
Helper          Killer